READ!!!

This is a first attempt at a classifier and turned out not to work well at all.

It basically converted the instructions, the replaced, and the replacement, into 3 w2v embeddings.
Even though it had acceptable scores on the validation set, it worked very very bad on unseen data.


In [9]:
import pandas as pd
from ast import literal_eval
survey_df = pd.read_csv(r'C:\Users\01din\PycharmProjects\thesis\data\survey_results\survey_results.csv')
generic = lambda x: literal_eval(x)
conv = {'nutrition' : generic, 'steps' : generic, 'ingredients' : generic, 'id_column' : generic, 'jaccard_similarity' : generic, 'diff' : generic, 'recipes' : generic, 'ingredients_original' : generic, 'tags': generic}
df = pd.read_csv(r"C:\Users\01din\PycharmProjects\thesis\data\cleaned_recipes\recipes_with_JS.csv", converters=conv)
df.drop(['Unnamed: 0'], inplace=True, axis=1)
df_ingredients = pd.read_csv(r"C:\Users\01din\PycharmProjects\thesis\data\ingredients\ingredients_nutrition.csv")
df_ingredients.drop(['Unnamed: 0'], inplace=True, axis=1)
df_ingredients['nutrition'] = df_ingredients['nutrition'].apply(lambda x: literal_eval(x) if pd.notnull(x) else x)

KeyboardInterrupt: 

In [2]:
ingredients = df_ingredients.ingredient.tolist()

In [ ]:
df['ingredients_original'] = df['ingredients_original'].apply(lambda x: ['_'.join(i.split()) for i in x])
df['steps'] = df['steps'].apply(lambda x: [f'<STEP> {s}' for s in x])
df['text'] = df.apply(lambda row: ' <INGR> '.join(row['ingredients_original']) + ' ' + ' '.join(row['steps']), axis=1)
from gensim.models import Word2Vec

# Prepare the corpus
corpus = df['text'].str.split().tolist()

# Train the Word2Vec model
model = Word2Vec(corpus, window=5, min_count=1, workers=4)

In [ ]:
df.text.iloc[0]

In [28]:
model.save(r"C:\Users\01din\PycharmProjects\thesis\models\w2v\recipe_word2vec.model")

In [11]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score

# Load the Word2Vec model
word2vec = Word2Vec.load(r'C:\Users\01din\PycharmProjects\thesis\models\w2v\recipe_word2vec.model')

# Function that calculates the average Word2Vec vector for a given piece of text
def average_word2vec(text, word2vec_model):
    words = text.split()
    word_vecs = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if not word_vecs:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word_vecs, axis=0)

# Calculate the average Word2Vec vectors for instructions, replaced, and replacement separately
df['instructions_vec'] = df['steps'].apply(lambda x: ' '.join(x)).apply(average_word2vec, args=(word2vec,))
survey_df['replaced_vec'] = survey_df['replaced'].apply(average_word2vec, args=(word2vec,))
survey_df['replacement_vec'] = survey_df['replacement'].apply(average_word2vec, args=(word2vec,))

# Concatenate the vectors
survey_df['concat_vec'] = survey_df.apply(lambda row: np.concatenate([row['replaced_vec'], row['replacement_vec'], df.loc[df['id'] == row['id'], 'instructions_vec'].values[0]]), axis=1)

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(survey_df['concat_vec'].tolist(), survey_df['result'], test_size=0.2, random_state=42)



# Initialize the classifier
clf = LogisticRegression()

# Define the hyperparameters and the values to test
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],  # inverse of regularization strength
    'penalty': ['l1', 'l2']  # l1 is Lasso, l2 is Ridge
}

# Define the scoring metric
scoring = make_scorer(accuracy_score)

# Define the cross-validation grid search
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring=scoring)

# Convert the list of vectors to a 2D array, which is necessary for sklearn
X_train_array = np.array(X_train)
y_train_array = np.array(y_train)

# Fit the model to the data
grid_search.fit(X_train_array, y_train_array)

# Get the best parameters
best_params = grid_search.best_params_

# Initialize and train the classifier using the best parameters
clf_best = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'])
clf_best.fit(X_train_array, y_train_array)

# Convert the list of test vectors to a 2D array
X_test_array = np.array(X_test)

# Predict on the test set
y_pred = clf_best.predict(X_test_array)

# Print the classification report
print(classification_report(y_test, y_pred))



C:\Users\01din\PycharmProjects\thesis\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\01din\PycharmProjects\thesis\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

              precision    recall  f1-score   support

          No       0.68      0.69      0.69        52
         Yes       0.69      0.67      0.68        52

    accuracy                           0.68       104
   macro avg       0.68      0.68      0.68       104
weighted avg       0.68      0.68      0.68       104



C:\Users\01din\PycharmProjects\thesis\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\01din\PycharmProjects\thesis\venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 70.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fit

In [12]:
clf = clf_best

In [13]:
def classify(input_data, word2vec_model, clf):
    # Compute the average Word2Vec vectors
    instructions_vec = average_word2vec(' '.join(input_data['instructions']), word2vec_model)
    replaced_vec = average_word2vec(input_data['replaced'], word2vec_model)
    replacement_vec = average_word2vec(input_data['replacement'], word2vec_model)

    # Concatenate the vectors
    concat_vec = np.concatenate([replaced_vec, replacement_vec, instructions_vec])

    # Reshape the vector to match the input shape of the classifier
    input_vector = concat_vec.reshape(1, -1)

    # Predict the class
    prediction = clf.predict(input_vector)

    return prediction

# Now you can classify a new instance like this:
input_data = {
    'instructions': df.iloc[0].steps,
    'replaced': 'yellow onions',
    'replacement': 'white onions'
}
result = classify(input_data, word2vec, clf)
print(result)


['No']


In [6]:
def get_matching_ingredients(df, ingredient):
    # Get the 'FdGrp_Desc' of the specific ingredient
    ingredient_grp = df[df['ingredient'] == ingredient]['FdGrp_Desc'].values[0]

    # Get all ingredients from the same group
    matching_ingredients = df[df['FdGrp_Desc'] == ingredient_grp]

    return matching_ingredients


def rank_replacements(instructions, replaced, possible_replacements, word2vec_model, clf):
    # possible_replacements = get_matching_ingredients(possible_replacements, replaced).ingredient.tolist()
    similar_words = word2vec_model.wv.most_similar(replaced, topn=30)
    possible_replacements = [word for word, similarity in similar_words]
    scores = []
    for replacement in possible_replacements:
        # Compute the feature vector for the replacement
        input_data = {
            'instructions': instructions,
            'replaced': replaced,
            'replacement': replacement
        }
        concat_vec = average_word2vec(' '.join(input_data['instructions']), word2vec_model)
        replaced_vec = average_word2vec(input_data['replaced'], word2vec_model)
        replacement_vec = average_word2vec(input_data['replacement'], word2vec_model)
        feature_vector = np.concatenate([replaced_vec, replacement_vec, concat_vec]).reshape(1, -1)

        # Get the classifier's confidence score for the replacement
        confidence_score = clf.predict_proba(feature_vector)[0][1]
        predicted_class = clf.predict(feature_vector)[0]
        scores.append((replacement, confidence_score, predicted_class))

    # Sort the replacements by the confidence scores in descending order
    scores.sort(key=lambda x: x[1], reverse=True)

    return scores
print(rank_replacements(df.iloc[0].steps, 'ground_beef', df_ingredients, word2vec, clf))

[('rump_roast', 0.3182893719958118, 'No'), ('cut_green_beans', 0.3149857359418725, 'No'), ('ground_venison', 0.3112943263634743, 'No'), ('hash_brown_potatoes', 0.3103270440281583, 'No'), ('lean_hamburger', 0.3097080993136041, 'No'), ('chuck_roast', 0.30927411343386313, 'No'), ('hamburger_meat', 0.3057874812346893, 'No'), ('ground_meat', 0.304608305804419, 'No'), ('v8_vegetable_juice', 0.30360360343243814, 'No'), ('bulk_italian_sausage', 0.3035349714686171, 'No'), ('ground_turkey_breast', 0.30245875209819983, 'No'), ('ground_sirloin', 0.2986165085000022, 'No'), ('ground_sausage', 0.2951283021459221, 'No'), ('lima_beans', 0.2947509172598281, 'No'), ('ground_round', 0.29239642895957396, 'No'), ('condensed_tomato_soup', 0.285662723150029, 'No'), ('lean_ground_turkey', 0.2851395617055952, 'No'), ('bulk_pork_sausage', 0.2838574873165043, 'No'), ('pork_sausage', 0.27845599810696564, 'No'), ('extra_lean_ground_beef', 0.2779598713018591, 'No'), ('italian_sausage', 0.26129130009537127, 'No'), ('

In [55]:
df.iloc[0]

id                                                                 112140
name                                            all in the kitchen  chili
n_ingredients                                                          13
ingredients_original    [ground_beef, yellow_onions, diced_tomatoes, t...
n_steps                                                                 6
steps                   [<STEP> <STEP> <STEP> brown ground beef in lar...
tags                    [time-to-make, course, preparation, main-dish,...
minutes                                                               130
description             this modified version of 'mom's' chili was a h...
nutrition                      [269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]
ingredients             [Beef,_ground,_70%_lean_meat_/_30%_fat,_raw, O...
contributor_id                                                     196586
submitted                                                      25/02/2005
similar                               

In [56]:
similar_words = word2vec.wv.most_similar('ground_beef', topn=30)